</div>
<B style="list-style-type: none; font-size: 33px; color: black; margin-top: 10;">Savings and IRA Funds Predictor for Retirees</B>
<br>
<div style="float: right; margin: 0px 15px 15px 0px;">
	<br><img src="https://comptroller.texas.gov/economy/fiscal-notes/2019/feb/images/pensionhero.jpg" width=400/>
</div>  <br>  <B style="color: #2B547E; display: block;">
<em style="color: teal;"> <strong>Business Challenge</strong></em>
<br><em style="color: teal;">Project by Gabriele Ferraina and Michele Pittarello</em><br>

</div>
<br><br><span style="color: teal;" </b> HULT International Business School <br></span>
<span style="color: teal;"</b> MBANDD Master in Business Analytics 
<br><br><div style="float: left; margin: 0px 15px 15px 0px;">
	<img src="https://upload.wikimedia.org/wikipedia/commons/b/b6/HULT_IBS_Logo_Outline_Black_%28cropped%29.png" width=250/>
</div>

In [1]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")  # This is to ignore any warnings that might pop up during execution

import matplotlib.pyplot as plt  #for plots
import numpy as np  #for numerical calculations
import pandas as pd #to work with data and manipulate it
import plotly.express as px #for plots
import seaborn as sns #for plots
from scipy import stats #for statistical analyses
import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

np.random.seed(42) #This is so every computation can be reproduced
pd.set_option('display.float_format','{:.2f}'.format)

These codes import the data into the file to avoid hardcoding numbers (manually inserting values)

Sources: 
*  Department of Treasury IRS (Nov 02, 2022). Tax Rate Schedules; Schedule Y-1 If your filling status is Married filling jointly or Qualifying Widow(er), U.S. IRS Federal Tax Rates, page 7; Retrieved on Dec 7th 2023 available at: http://www.irs.gov/pub/irs-pdf/f1040es.pdf
*  State of California Franchise Tax Board (2022). CALIFORNIA WITHHOLDING SCHEDULES FOR 2023. 2023 Withholding Schedules - Method B (INTERNET) Page 7. Retrieved on Dec 12th 2023. Available at https://edd.ca.gov/siteassets/files/pdf_pub_ctr/23methb.pdf
*  Massachusetts Department of Revenue (Feb 02, 2023). Personal Income Tax for Residents. Retrieved on Dec 7th 2023 available at https://www.mass.gov/info-details/massachusetts-tax-rates
*  Internal Revenue Service (IRS). (2023). Publication 590-B (2022), Distributions from Individual Retirement Arrangements (IRAs) | Internal Revenue Service. IRS. https://www.irs.gov/publications/p590b

In [2]:
import_path = "./data/"

import_filename = "californiatax.csv"

import_data = pd.read_csv(import_path + import_filename)

californiatax = import_data.copy()

import_filename = "federaltax.csv"

import_data = pd.read_csv(import_path + import_filename)

federaltax = import_data.copy()

import_filename = "inflation.csv"

import_data = pd.read_csv(import_path + import_filename)

inflation = import_data.copy()

import_filename = "irsira.csv"

import_data = pd.read_csv(import_path + import_filename)

irsira = import_data.copy()

These codes create functions that carry out all necessary calculations

In [3]:
#taxes
def calculate_federal_tax(income):
    basetax=federaltax.iloc[federaltax[federaltax['Over']<=income].last_valid_index()]['Base Tax']
    taxpercentage=federaltax.iloc[federaltax[federaltax['Over']<=income].last_valid_index()]['Tax Percentage']
    amountfrombasetax=federaltax.iloc[federaltax[federaltax['Over']<=income].last_valid_index()]['Over']
    taxes=(income-amountfrombasetax)*taxpercentage + basetax
    return taxes

def calculate_california_tax(income):
    basetax=californiatax.iloc[californiatax[californiatax['Over']<=income].last_valid_index()]['Base Tax']
    taxpercentage=californiatax.iloc[californiatax[californiatax['Over']<=income].last_valid_index()]['percentage']
    amountfrombasetax=californiatax.iloc[californiatax[californiatax['Over']<=income].last_valid_index()]['Over']
    taxes=(income-amountfrombasetax)*taxpercentage + basetax
    return taxes

def total_tax_cali(income):
    taxcali=calculate_california_tax(income)
    taxfed=calculate_federal_tax(income)
    taxes=taxcali+taxfed
    return taxes

def total_mass_tax(income):
    taxfed=calculate_federal_tax(income)
    taxmass=income*0.05
    taxes=taxmass+taxfed
    return taxes

#final table
def editmergedtable():
    global yearlyexpense, starting_year, savingsbalance, ratesavings, irabalance, rateira, age, inflation, dates, datestable, mergedtable
    age = agebutton.value
    starting_year = current_year.value
    yearlyexpense = int(yearly_expense.value)
    savingsbalance = int(moneyinsavings.value)
    irabalance = int(moneyinirasavings.value)
    rateira = irainterest.value / 100.0  # Convert to percentage
    ratesavings = savingsinterest.value / 100.0  # Convert to percentage
    inflation["Annual Expenses"]=inflation["Inflation Multiplier"] * yearlyexpense 
    starting_year = int(starting_year)
    dates = range(starting_year, inflation["Year"].max()+1)
    datestable = pd.DataFrame(dates, columns=['Year'])
    mergedtable = datestable.merge(inflation[['Year', 'Annual Expenses']], on='Year')
    mergedtable['Age']=age-int(starting_year)+mergedtable['Year']
    mergedtable=mergedtable.merge(irsira[['Age','Distribution Period in Years']], on='Age')
    mergedtable['Annual Expenses with previous year taxes']=0
    mergedtable['Minimum IRA Withdrawals']=0
    mergedtable['Withdrew IRA Balance']=0
    mergedtable['Expenses after IRA Withdrawals']=0
    mergedtable['Savings Interest']=0
    mergedtable['Savings']= 0
    mergedtable['Savings'][0]=savingsbalance
    mergedtable['Expenses after Savings']=0
    mergedtable['IRA Balance']=0
    mergedtable['IRA Interest']=0
    mergedtable['IRA Balance'][0]=irabalance
    mergedtable['Year Withdrawn']=0
    mergedtable['MA Tax']=0
    mergedtable['CA Tax']=0
    mergedtable['Tax Savings']=0
    mergedtable['Ideal State']=0
    diff= mergedtable['Year'].max()-starting_year
    i=0
    while i<=diff:
        mergedtable['Savings Interest'][i]=mergedtable['Savings'][i]*ratesavings
        mergedtable['IRA Interest'][i]=mergedtable['IRA Balance'][i]*rateira
        if i==0:
            mergedtable['Annual Expenses with previous year taxes'][i]=mergedtable['Annual Expenses'][i]
        
        if mergedtable['Distribution Period in Years'][i]>0 and mergedtable['IRA Balance'][i]>0 :
            mergedtable['Minimum IRA Withdrawals'][i]=mergedtable['IRA Balance'][i]/mergedtable['Distribution Period in Years'][i]
        else:
            mergedtable['Minimum IRA Withdrawals'][i]=0
        
        if i==0:
            mergedtable['Withdrew IRA Balance'][i]= mergedtable['Minimum IRA Withdrawals'][i]
        else:
            mergedtable['Withdrew IRA Balance'][i]=mergedtable['Minimum IRA Withdrawals'][i]+mergedtable['Withdrew IRA Balance'][i-1]
        
        if mergedtable['Withdrew IRA Balance'][i]>=mergedtable['Annual Expenses with previous year taxes'][i]:
            mergedtable['Expenses after IRA Withdrawals'][i]=0
            mergedtable['Withdrew IRA Balance'][i]=mergedtable['Withdrew IRA Balance'][i]-mergedtable['Annual Expenses with previous year taxes'][i]
        else:
            mergedtable['Expenses after IRA Withdrawals'][i]=mergedtable['Annual Expenses with previous year taxes'][i]-mergedtable['Withdrew IRA Balance'][i]
            mergedtable['Withdrew IRA Balance']=0
        
        if mergedtable['Savings Interest'][i]-mergedtable['Expenses after IRA Withdrawals'][i]+mergedtable['Savings'][i]>0:
            mergedtable['Savings'][i+1]=mergedtable['Savings Interest'][i]-mergedtable['Expenses after IRA Withdrawals'][i]+mergedtable['Savings'][i]
            mergedtable['Expenses after Savings'][i]=0
        
        else: 
            mergedtable['Savings'][i+1]=0
            mergedtable['Expenses after Savings'][i]=mergedtable['Expenses after IRA Withdrawals'][i]-mergedtable['Savings'][i]-mergedtable['Savings Interest'][i]
        
        if mergedtable['Minimum IRA Withdrawals'][i]+mergedtable['Expenses after Savings'][i]<=mergedtable['IRA Balance'][i]:
            mergedtable['Year Withdrawn'][i]=mergedtable['Minimum IRA Withdrawals'][i]+mergedtable['Expenses after Savings'][i]
            mergedtable['IRA Balance'][i+1]=mergedtable['IRA Balance'][i]+mergedtable['IRA Interest'][i]-(mergedtable['Year Withdrawn'][i])
        else:
            mergedtable['Year Withdrawn'][i]=mergedtable['Minimum IRA Withdrawals'][i]+mergedtable['IRA Balance'][i]
            mergedtable['IRA Balance'][i+1]=0
        
        
        mergedtable['MA Tax'][i]= total_mass_tax(mergedtable['Year Withdrawn'][i]+mergedtable['Savings Interest'][i])
        mergedtable['CA Tax'][i]= total_tax_cali(mergedtable['Year Withdrawn'][i]+mergedtable['Savings Interest'][i])
        appliedtax=min(mergedtable['MA Tax'][i],mergedtable['CA Tax'][i])
        mergedtable['Tax Savings'][i]=max(mergedtable['MA Tax'][i],mergedtable['CA Tax'][i])-min(mergedtable['MA Tax'][i],mergedtable['CA Tax'][i])
        
        if mergedtable['MA Tax'][i]>mergedtable['CA Tax'][i]:
            mergedtable['Ideal State'][i]='CA'
        else: mergedtable['Ideal State'][i]='MA'
            
            
            
        
        if i==0:
            mergedtable['Annual Expenses with previous year taxes'][i+1]=mergedtable['Annual Expenses'][i+1]+appliedtax
        elif i<diff:
            mergedtable['Annual Expenses with previous year taxes'][i+1]=mergedtable['Annual Expenses'][i+1]+appliedtax
        else:
            mergedtable['Annual Expenses with previous year taxes'][i+1]=0

        i=i+1
    
        
def savingssrunouton():
    global mergedtable
    try:
        yeardepletion = "Your Savings will be depleted after year " + str(mergedtable.query('Savings == 0').iloc[0]['Year'])
        return yeardepletion
    except IndexError:
        lastdate = mergedtable['Year'].max()
        yeardepletion = "Your Savings will be depleted after after last date available in study: " + str(lastdate)
        return yeardepletion

    
def irasrunouton():
    global mergedtable
    try:
        yeardepletion = "Your IRA Funds will be depleted after year " + str(mergedtable[mergedtable['IRA Balance']<=0].iloc[0]['Year'])
        return yeardepletion
    except IndexError:
        lastdate = mergedtable['Year'].max()
        yeardepletion = "Your IRA Funds will be depleted after after last date available in study: " + str(lastdate)
        return yeardepletion

def agerunout():
    global mergedtable
    try:
        lastage = "Your age when your funds finish is " + str(mergedtable[mergedtable['IRA Balance']<=0].iloc[0]['Age'])
        return lastage
    except IndexError:
        lastdate = mergedtable['Age'].max()
        lastage = "Your IRA Funds will last after you reach age: " + str(lastdate) + ". There is not enough data to predict after"
        return lastage

The following codes use functions and widgets to create visualizations and a user interface.

In [4]:
#button setups
button_layout = widgets.Layout(width='600px', height='100px')  # Adjust width and height as needed
button_style = widgets.ButtonStyle(button_color='lightblue')
button_style2 = widgets.ButtonStyle(button_color='lightgreen')
text_layout = widgets.Layout(width='600px', height='20px')  
description_width = 'initial'
btn_getfunds=widgets.Button(description="SEE FUNDS", layout=button_layout, style=button_style)

getresult=widgets.Button(description="VERIFY VALUES", layout=button_layout, style=button_style)
btn_back1=widgets.Button(description="GO BACK", layout=button_layout, style=button_style)
btn_back2=widgets.Button(description="GO BACK", layout=button_layout, style=button_style)
btn_proceed = widgets.Button(description="CHECK RESULTS", layout=button_layout, style=button_style)
btn_start = widgets.Button(description="START", layout=button_layout, style=button_style)
btn_taxschedule = widgets.Button(description="VIEW YOUR TAX SCHEDULE", layout=button_layout, style=button_style)

In [5]:
#to visualize the total funds over time
def getfundsgraph(self):
    clear_output()

    long_format = mergedtable.melt(id_vars='Year', var_name='Category', value_name='Amount')
    filtered_data = long_format[long_format['Category'].isin(['Annual Expenses', 'IRA Balance', 'Savings'])]

    zero_balances = mergedtable[(mergedtable['IRA Balance'] == 0)]
    if not zero_balances.empty:
        last_year = zero_balances['Year'].min()
        filtered_data = filtered_data[filtered_data['Year'] <= last_year]

    fig2 = px.line(filtered_data, x='Year', y='Amount', color='Category', labels={'Amount': 'USD'})
    fig2.update_layout(title='Annual Expenses, IRA Balance, and Savings Over Years', xaxis_title='Year', yaxis_title='USD')
    display(fig2, btn_back2)


In [6]:
#to visualize the tax schedule and ideal state to live in based on a specific year
def gettaxschedule(self):
    clear_output()
    fig = px.bar(mergedtable[mergedtable['IRA Balance'] > 0], x='Year', y='Tax Savings', color='Ideal State',
             hover_data=mergedtable[mergedtable['IRA Balance'] > 0][['Tax Savings']], 
             labels={'Tax Savings': 'Sum of Tax Savings'}, 
             barmode='group')

    fig.update_layout(
        title="Yearly Tax Savings Grouped by Ideal State",
        xaxis_title="Year",
        yaxis_title="Sum of Tax Savings"
    )

    display(fig, btn_back2 )
    

In [7]:
#to visualize all informations that are useful for the pensioner
def ThirdPage(self):
    clear_output()
    date1= savingssrunouton()
    date2 = irasrunouton()
    age1= agerunout()
    btn_taxschedule.on_click(gettaxschedule)
    btn_savingsend=widgets.Button(description=date1, layout=button_layout, style=button_style2)
    btn_iraend=widgets.Button(description=date2, layout=button_layout, style=button_style2)
    btn_lastage=widgets.Button(description=age1, layout=button_layout, style=button_style2)
    display(btn_savingsend,btn_iraend,btn_lastage,btn_getfunds, btn_taxschedule, btn_back1)
    
    

In [8]:
#sets up a page for value insertion and an intermediary page two that verifies inputs
def FirstPage(self):
    clear_output()

    display (agebutton,current_year, yearly_expense, moneyinsavings, moneyinirasavings,savingsinterest, irainterest, getresult)


agebutton = widgets.IntSlider(
    value=0,
    min=0,
    max=120,
    step=1,
    description='Age:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='600px', height='20px') 
)

current_year = widgets.IntSlider(
    value=0,
    min=inflation['Year'].min(),
    max=inflation['Year'].max(),
    step=1,
    description='Current Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='600px', height='20px') 
)

yearly_expense = widgets.Text(
    placeholder='What is your current yearly expense',
    description='Type your yearly expense as a number',
    disabled=False,
    layout=text_layout,
    style={'description_width': description_width}
)

moneyinsavings = widgets.Text(
    placeholder='What is your current Savings Account Balance',
    description='Type your Savings Account Balance as a number',
    disabled=False,
    layout=text_layout,
    style={'description_width': description_width}
)

moneyinirasavings = widgets.Text(
    placeholder='What is your current IRA Account Balance',
    description='Type your IRA Account Balance as a number',
    disabled=False,
    layout=text_layout,
    style={'description_width': description_width}
)

irainterest = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=0.5,
    description='IRA Interest Rate %:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px', height='20px') 
)

savingsinterest = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=0.5,
    description='Savings Account Interest Rate %:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px', height='20px') 
)


def SecondPage(self):

    if (agebutton.value == 0 or 
        current_year.value == 0 or 
        yearly_expense.value.strip() == '' or 
        moneyinsavings.value.strip() == '' or 
        moneyinirasavings.value.strip() == '' or 
        irainterest.value == 0 or 
        savingsinterest.value == 0):
        print('Error: Please fill in all fields with valid information.')
        display(btn_back1)
        
    else:
        try:
            currentage = agebutton.value
            starting_year = current_year.value
            yearlyexpense = int(yearly_expense.value)
            savingsbalance = int(moneyinsavings.value)
            irabalance = int(moneyinirasavings.value)
            rateira = irainterest.value / 100.0  # Convert to percentage
            ratesavings = savingsinterest.value / 100.0  # Convert to percentage
            editmergedtable()
            display(btn_proceed)
            
            

        except ValueError:
            print("Error: Please make sure all numerical fields contain only numbers.")
            display(btn_back1)



In [9]:
#button actions
btn_proceed.on_click(ThirdPage)
getresult.on_click(SecondPage)
btn_back1.on_click(FirstPage)
btn_start.on_click(FirstPage)
btn_taxschedule.on_click(gettaxschedule)
btn_back2.on_click(ThirdPage)
btn_getfunds.on_click(getfundsgraph)

In [10]:
#executes the program
display(btn_start)

Button(description='GO BACK', layout=Layout(height='100px', width='600px'), style=ButtonStyle(button_color='li…